# Dimensionality Reduction


## Agenda
- Why do we consider dimensionality reduction techniques? 
- PCA. 
- Kernel PCA.  
- LDA.  
- Manifold with t-SNE.   


Generally following `Machine Learning with Python 3rd Edition, Chapter 5`.

- Also known as feature extraction.  
- Maps the original feature space into a lower subspace with lower dimensionality than the original one.  
- This lower dimensional subspace should be less sparse, have features that are less correlated with each other, and explain a significant amount of the original variance in the full feature space.

### Potential Benefits
- Less data to deal with.  
- Less spareness.  
- Helps with overfitting and bias issues.  
- Can leverage compression to be able to [visualize high-dimensional data](https://projector.tensorflow.org) in lower dimensions that retain characteristics of the overall variance.  
- Can make the data more conducive to modeling with kernel transformations.  
- May be able to visualize the high-dimensional data in 2 or 3-dimension space.

### Cons  
- Not as intrepretable.  
- Some compression methods can be computationally intensive.  
- This simplifies the features, so there is some amount of information loss that simple models may need, but other, more complex models may be better using the raw data.

> Think of this as compression, essentially the same thing as compressing a large file into a .zip file. The file is still there, but it has been managed to be compressed to a smaller file size.

This can also help with the `curse of dimensionality`. Since we will be compressing the space and reducing the spareness, it should squeeze the data closer together in the new subspace.

The `curse of dimensionality` can make training slow since we have may have a large, sparse space and it can make finding solutions difficult.  
- In a 1 x 1 box, 0.4% of random points would be within 0.001 from one of the borders, i.e., extreme observations along one of the dimensions.  
- In a 10,000 x 10,000 space, nearly 100% of the observations would be near a border, i.e., across enough dimensions, almost every point will be extreme if you considered all the dimensions.

- If you pick two point at random on a unit square, the distance will be on average 0.52 units away.  
- If you pick two point at random on a 3-D cube, the distance will be on average 0.66 units away.  
- In a 1,000,000-dimension hypercube, the average distance is about 408 units.

> Since points will be so much further away in higher dimensions, the predictions will be less reliable than in lower dimensions.

### Visualization example: 3-dimensional data lying near a 2-dimensional plane
Think of a dataset with 3 continuous features, and one of the features being essentially worthless in terms of its value to separating the data.

<img src='./diagrams/3d-data-close-to-2d.png'>

[Image source: Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow](https://github.com/ageron/handson-ml2)

### After projection to a 2-dimensional space

<img src='./diagrams/2d-data-post-projection.png'>

[Image source: Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow](https://github.com/ageron/handson-ml2)

# Not everything works with simple projections

### Swiss Roll
[See scikit-learn's datasets if you want to examine the data.](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_swiss_roll.html)

<img src='./diagrams/swiss-roll.png'>

[Image source: Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow](https://github.com/ageron/handson-ml2)

#### Projecting versus unrolling
Left shows a projection similar to the 3-dimensional to 2-dimensional example. The right shows a different technique that looks at the manifold and tries to learn the shape.

<img src='./diagrams/swiss-unroll.png'>

[Image source: Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow](https://github.com/ageron/handson-ml2)

# Principal Components (PCA)
PCA is a projection technique. In many cases data is not spread uniformly across all dimensions, it'll be concentrated among a subset like the 3-dimensional to 2-dimensional example above. We can project the dataset to a lower dimensional subspace without losing much of the information. Projections won't work well for data like the `swiss roll` example since the data may twist and turn and will require a [manifold](https://en.wikipedia.org/wiki/Manifold) technique.

- Is an unsupervised learning modeling.   
- Very common method and has been around since 1901.  
- Helps with dimensionality reduction and remove of noise in the data.  
- Commonly used as a preprocessing step. 
- Shouldn't be used blindly or purely to regularize your data, i.e., you should have just cause for using it.  

<img src='./diagrams/pca.png' style='width: 600px'>

### Basic Idea
- Finds directions of maximum variance in high-dimension space and projects it to a lower space with the same or fewer dimensions.  
- The components are orthogonal axes of the subspace and are the directions of maximum variance.
- We construct a $d \times k-dimensional$ transformation matrix, $W$, that maps the vector, $x$, the features of the training data to the new subspace. The new subspace will have fewer dimensions then the original $d$ dimensions.  
- It starts with looking at the hyperplane that lies closest to the data, and projects the data to it.

$$
x=[x_1, x_2, \dots, x_d], x \in \mathbb{R}^d
$$

That is transformed by a matrix, $W \in \mathbb{R}^{d \times k}$:

$$
xW=z, \space \text{where } z = [z_1, z_2, \dots, z_k], z \in \mathbb{R}^k
$$

$d$-dimensional data is now projected to $k$-dimesions, generally $k<<d$.
- The first principal component will explain the largest amount of variance, the second the second most variance, ...  
- Very sensitive to scaling, so you'll need to standardize the data prior. If you don't the features will have different relative importances, which we probably don't want.  
- As noted earlier, the components are uncorrelated, even if the features are correlated. This can be handy for dealing with multicollinearity issues.

> Minor issue: small differences in the training set, e.g., you find more examples or update a feature, could cause different results when you run PCA.

### Steps in the Process
- Standardize the feature matrix (`min-max` or `standardize`).  
- Create covariance matrix.  
- Decompose into eigenvectors and eigenvalues.  
- Sort eigenvalues by decreasing order to rank the eigenvectors.  
- Select $k$ eigenvectors. This is the primary tuning knob.  
- Construct the projection matrix.  
- Transform the data.  

In [ ]:
import pandas as pd

wine = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/ensembles/wine.data', header=None)
wine.info()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

pd.plotting.scatter_matrix(wine.iloc[:,1:], figsize=(9,9))
plt.show()

> Many features that at least visually seem to be highly correlated.

#### Step 1: Split and Scale data

In [ ]:
from sklearn.model_selection import train_test_split

X, y = wine.iloc[:, 1:].values, wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    stratify=y, 
                                                    random_state=0
                                                   )

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std  = sc.transform(X_test)

#### Step 2: Covariance Matrix
Pairwise covariances between features $j$ and $k$.

$$
\sigma_{jk} = \frac{1}{n-1}\sum{_{i=1}^n}(x_j^{(i)}-\mu_k)
$$

Since the data is standardized, $\mu_k = 0$, so a positive covariance means the features increase together.

In [ ]:
import numpy as np

cov_mat = np.cov(X_train_std.T)
print(cov_mat.round(1))

#### Step 3: Eigenpairs of Covariance Matrix
Extracted the 13 eigenvalues, same as $d$.

In [ ]:
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

plt.plot(eigen_vals)
plt.title('Eigenvalues', loc='left')
plt.show()

In [ ]:
eigen_vecs
eigen_vals

#### Step 4: Sort & Step 5: Selecting the meaning $k$ Eigenvalues
- Eigenvalues define the magnitude of the eigenvector, so they need to be sorted.  
- Want to select the top $k$, usually based on how much variance they explain.

$$
\text{Explained variance ratio} = \frac{\lambda_j}{\sum \lambda_j}
$$

In [ ]:
tot = sum(eigen_vals)

var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

plt.bar(range(1,14), var_exp, alpha=0.5, align='center', label='Variance Explained')
plt.step(range(1,14), cum_var_exp, where='mid', label='Cumulative Explained')
plt.xlabel('Principal Componment')
plt.ylabel('Explained Variance')
plt.show()

- First component explains nearly 40% of the variance in the data.  
- First and second explain almost 60%. So we'll set $k=2$.  
- Altogether they will sum to 100%.

- Note: PCA is an unsupervised method which means that information about the class is ignored.

#### Step 6: Projection Matrix 

In [ ]:
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i]) for i in range(len(eigen_vals))]
eigen_pairs.sort(key=lambda k: k[0], reverse=True)

Extract first $2$ eigenvectors:

In [ ]:
w = np.hstack((eigen_pairs[0][1][:, np.newaxis],
              eigen_pairs[1][1][:, np.newaxis]))

print(w)

#### Step 7: Transform
$$
X'=XW
$$

In [ ]:
X_train_std

In [ ]:
X_train_pca = X_train_std.dot(w)
X_train_pca

In [ ]:
plt.plot(X_train_pca[:,0], X_train_pca[:,1], 'bo')
plt.show()

### Reconstruction

$$
X_{recovered}=X_{d-projection}W_{d}^T
$$

### Using within a Model
- $k$ can be treated as a hyperparameter.  
- Use GridSearch to determine the best value for $k$ - usually searching around 90-95% of explained variance is the best starting point.  
- You could also look at the explained variance plot and look for an elbow, but that isn't going to be as quantitatively based and may not maximize predictive performance.

In [ ]:
wine.head(1)

In [ ]:
wine.iloc[:, 0].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wine.iloc[:, 1:],
                                                   wine.iloc[:, 0],
                                                   test_size=0.20
                                                   )

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

p = Pipeline([('scaling', StandardScaler()), 
              ('pca', PCA()),
              ('model', LogisticRegression())
             ])

params = {'model__C': [0.01, 0.1, 1, 10], 'pca__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

wine_search = GridSearchCV(p, param_grid=params, scoring='accuracy', cv=10, refit=True)
wine_search = wine_search.fit(X_train, y_train)

wine_search.best_estimator_

In [ ]:
print(f'Validation score: {wine_search.best_score_:.2%}')
print(f'Test score: {wine_search.score(X_test, y_test):.2%}')

### Using with Visualization
If using to visualize the data, it will only make sense to look at 2 or 3 components.

In [ ]:
X, y = wine.iloc[:, 1:], wine.iloc[:, 0]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

p = Pipeline([('scaling', StandardScaler()), 
              ('pca', PCA(n_components=3))
             ])

new_X = p.fit_transform(X)
new_X = pd.DataFrame(new_X)
new_wine = pd.concat([new_X, pd.Series(y)], axis=1)
new_wine.columns = ['comp1', 'comp2', 'comp3', 'class']
new_wine['class'] = new_wine['class'].astype('str')
new_wine.head()

In [ ]:
import seaborn as sns

sns.scatterplot(x='comp1', y='comp2', hue='class', data=new_wine)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.show()

> Can help visualize if the 1st and 2nd (and/or 3rd) have a high cumulative explained variance ratio, otherwise, may not help if you have trying to visualize distance between classes.

## Example with MNIST

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, parser='auto')
mnist.keys()

In [ ]:
mnistX = mnist['data']
mnistX.shape

In [ ]:
mnistY = mnist['target']
mnistY.shape

#### Sample to make this run faster

In [ ]:
samples = 2500

mnist_sample = mnist.data.sample(2500)
mnist_sample_targets = mnist.target.iloc[mnist_sample.index.tolist()]

### See if PCA is beneficial for reducing the space
Original space is 784 dimensions.

In [ ]:
# will take a couple minutes to run.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mnist_sample,
                                                   mnist_sample_targets,
                                                   test_size=0.20
                                                   )

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

p = Pipeline([('scaling', StandardScaler()), 
              ('pca', PCA()),
              ('model', LogisticRegression(solver='liblinear'))
             ])

params = {'model__C': [0.01, 10, 100], 'pca__n_components': [25, 50, 100]}

mnist_search = GridSearchCV(p, param_grid=params, scoring='accuracy', cv=10, refit=True)
mnist_search = mnist_search.fit(X_train, y_train)

mnist_search.best_estimator_

In [ ]:
print(f'Validation score: {mnist_search.best_score_:.2%}')
print(f'Test score: {mnist_search.score(X_test, y_test):.2%}')

> Perform a secondary search to find "better" hyperparameter values. We won't do this with every example, but recall this is best practice for thoroughness. We could scan every possible combinations initially, but that will take a long time...

In [ ]:
params = {'model__C': [5, 10, 15], 'pca__n_components': [40, 50, 60]}

mnist_search = GridSearchCV(p, param_grid=params, scoring='accuracy', cv=10, refit=True)
mnist_search = mnist_search.fit(X_train, y_train)

mnist_search.best_estimator_

In [ ]:
print(f'Validation score: {mnist_search.best_score_:.2%}')
print(f'Test score: {mnist_search.score(X_test, y_test):.2%}')

#### MNIST Compression

In [ ]:
X_train.head()

In [ ]:
pca = PCA(n_components=50)
X_reduced = pca.fit_transform(X_train)
X_reduced.shape

In [ ]:
X_recovered = pca.inverse_transform(X_reduced)
X_recovered.shape

In [ ]:
def plot_digits(instances, images_per_row=5, **options):
    '''
    From: https://github.com/ageron/handson-ml2/blob/master/08_dimensionality_reduction.ipynb
    '''
    size = 28
    images_per_row = min(len(instances), images_per_row)

    # This is equivalent to n_rows = ceil(len(instances) / images_per_row):
    n_rows = (len(instances) - 1) // images_per_row + 1

    # Append empty images to fill the end of the grid, if needed:
    n_empty = n_rows * images_per_row - len(instances)
    padded_instances = np.concatenate([instances, np.zeros((n_empty, size * size))], axis=0)

    # Reshape the array so it's organized as a grid containing 28×28 images:
    image_grid = padded_instances.reshape((n_rows, images_per_row, size, size))

    # Combine axes 0 and 2 (vertical image grid axis, and vertical image axis),
    # and axes 1 and 3 (horizontal axes). We first need to move the axes that we
    # want to combine next to each other, using transpose(), and only then we
    # can reshape:
    big_image = image_grid.transpose(0, 2, 1, 3).reshape(n_rows * size,
                                                         images_per_row * size)
    # Now that we have a big image, we just need to show it:
    plt.imshow(big_image, **options)
    plt.axis("off")
    
plt.figure(figsize=(10, 10))
plt.subplot(121)
plot_digits(X_train[::20])
plt.title("Original", fontsize=16)
plt.subplot(122)
plot_digits(X_recovered[::20])
plt.title("Compressed", fontsize=16)
plt.show()

### Wrapping up PCA
- PCA can be useful reducing the dimensions without significantly loss of information. 
- Only do when necessary (try regularization first).  
- The number of components needs to be treated as a hyperparameter.  
- If there are a lot of features, searching around 90-99% of explained variance ratio usually ideal.  
- PCA uses singular value decomposition (SVD), but scikit-learn has solvers that approximate SVD that are dramatically faster. It generally uses this `randomized` solver, but it may use the `full` SVD.  
- There is a variant, `Incremental PCA` that allows for only part of the dataset to be loaded at once. We'll talk about this in a couple weeks, but it is an option for larger datasets.  
- For sparse datasets, see [SparsePCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html).

# Kernel PCA
Maps samples into a higher dimensional space, which allows for non-linear transformations, which we've seen previously with kernels in support vector machines.

<img src='./diagrams/sklearn-kernel-pca.png' style='width: 600px'>

[Image source: scikit-learn](https://scikit-learn.org/stable/modules/decomposition.html#kernel-pca)


See [Scholkopf's notes on *Kernel Principal Component Analysis*](http://pca.narod.ru/scholkopf_kernel.pdf)

- While PCA and KernelPCA are unsupervised, KernelPCA doesn't have some of the features as PCA.  
- No explained variance methods, so we have to rely on predictive performance and hyperparameter tuning.  
- Best to use this as a step within a pipeline and try different kernels and components extracted during grid search.  
- Another option is to look at the reconstruction / compression error and pick the hyperparameters that offer the lowest reconstruction error. This would be similar to the above example using the MNIST data.  

> Recall that kernel tricks will be a bit more computationally intensive and may not increase performance. Unless the data is nonlinear and would benefit from higher order transformations, it may be of little use.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import KernelPCA

p = Pipeline([('scaling', StandardScaler()), 
              ('kpca', KernelPCA()),
              ('model', LogisticRegression(solver='liblinear'))
             ])

params = {'model__C': [0.01],
          'kpca__n_components': [10, 100, 150],
          'kpca__kernel': ['poly', 'rbf']
         }

mnist_search = GridSearchCV(p, param_grid=params, scoring='accuracy', cv=10, refit=True)
mnist_search = mnist_search.fit(X_train, y_train)

mnist_search.best_estimator_

In [ ]:
print(f'Validation score: {mnist_search.best_score_:.2%}')
print(f'Test score: {mnist_search.score(X_test, y_test):.2%}')

> Not much improvement in the MNIST data. We would see a pronounced difference if the data was similar to the first figure in this section, i.e., highly non-linear.

> Remember, we'll want to do at least a secondary pass on the hyperparameters for complete due diligence.

In [ ]:
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

X, y = make_circles(n_samples=1_000, factor=0.3, noise=0.05, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [ ]:
import matplotlib.pyplot as plt

_, (train_ax, test_ax) = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(8, 4))

train_ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train)
train_ax.set_ylabel("Feature #1")
train_ax.set_xlabel("Feature #0")
train_ax.set_title("Training data")

test_ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
test_ax.set_xlabel("Feature #0")
_ = test_ax.set_title("Testing data")

In [ ]:
from sklearn.decomposition import PCA, KernelPCA

pca = PCA(n_components=2)
kernel_pca = KernelPCA(
    n_components=2, 
    kernel="rbf", 
    gamma=10, 
)

X_test_pca = pca.fit(X_train).transform(X_test)
X_test_kernel_pca = kernel_pca.fit(X_train).transform(X_test)
fig, (orig_data_ax, pca_proj_ax, kernel_pca_proj_ax) = plt.subplots(
    ncols=3, figsize=(14, 4)
)

orig_data_ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
orig_data_ax.set_ylabel("Feature #1")
orig_data_ax.set_xlabel("Feature #0")
orig_data_ax.set_title("Testing data")

pca_proj_ax.scatter(X_test_pca[:, 0], X_test_pca[:, 1], c=y_test)
pca_proj_ax.set_ylabel("Principal component #1")
pca_proj_ax.set_xlabel("Principal component #0")
pca_proj_ax.set_title("Projection of testing data\n using PCA")

kernel_pca_proj_ax.scatter(X_test_kernel_pca[:, 0], X_test_kernel_pca[:, 1], c=y_test)
kernel_pca_proj_ax.set_ylabel("Principal component #1")
kernel_pca_proj_ax.set_xlabel("Principal component #0")
_ = kernel_pca_proj_ax.set_title("Projection of testing data\n using KernelPCA")

In [ ]:
p = Pipeline([('scaling', StandardScaler()), 
              ('kpca', KernelPCA()),
              ('model', LogisticRegression(solver='liblinear'))
             ])

params = {
        'model__C': [0.01],
        'kpca__gamma': [None, 2, 10, 50],
        'kpca__kernel': [ 'rbf']
         }

circle_search = GridSearchCV(p, param_grid=params, scoring='accuracy', cv=10, refit=True)
circle_search = circle_search.fit(X_train, y_train)

circle_search.best_estimator_

In [ ]:
print(f'Validation score: {circle_search.best_score_:.2%}')
print(f'Test score: {circle_search.score(X_test, y_test):.2%}')

# Linear Discriminant Analysis (LDA)
- Similar in concept to PCA and will have many of the same methods, e.g., `explained_variance_ratio_`.  
- Actually a supervised learning algorithm.  
- Looks at finding axes (hyperplanes) that discriminant the most between classes.  
- Unlike PCA, this keeps the classes as far apart as possible, which isn't a trait of PCA.  
- Though you'd assume this may work better than PCA, generally PCA is a better option.

<img src='./diagrams/lda5-6.png' style='width: 600px'>

[Image source Machine Learning with Python 3rd Edition, page 160](https://github.com/rasbt/python-machine-learning-book-3rd-edition/tree/master/ch05/images)

- LDA assumes the data is normally distributed.  
- In the above, `LD 1` would separate the classes well.  
- `LD 2`wouldn't be a good discriminant since it wouldn't separate the data if you were able to draw a horizontal line on the y-axis.  
- Similar to linear regression, it assumes independence between the samples, which is really a soft requirement, as it can still work reasonably well if that is violated.

### Basic Steps with LDA
- Standardize the $d$-dimensional dataset.  
- Compute the $d$-dimensional mean vector.  
- Create between-class scatter matrix, $S_B$ and within-class scatter matrix, $S_w$.  
- Calculate [eigenvectors](https://mathworld.wolfram.com/Eigenvector.html) and [eigenvalues](https://mathworld.wolfram.com/Eigenvalue.html) for $S_{W}^{-1}S_B$.  
- Sort the eigenvalues to rank eigenvectors.  
- Pick $k$ eigenvectors and create a $d\times k$-dimensional matrix, $W$.  
- Project examples using $W$.

> If there is perfect collinearity, only one component will be returned. This should be very rare and more a byproduct of a potential error in your analysis.

Similar steps to PCA. The class label comes into play with the mean vector.

See [Machine Learning with Python 3rd Edition, Pages 161-164 for manual implementation](https://github.com/rasbt/python-machine-learning-book-3rd-edition/tree/master/ch05).

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(wine.iloc[:, 1:],
                                                   wine.iloc[:, 0],
                                                   test_size = 0.2
                                                   )

ss = StandardScaler()
x_train_scale = ss.fit_transform(X_train)

lda = LDA(n_components=2)
x_train_lda = lda.fit_transform(x_train_scale, y_train)

x_train_lda[:5, :]

In [ ]:
lda.explained_variance_ratio_

> Outputs components and their explained variance ratios, similar to PCA

In [ ]:
lda.score(x_train_scale, y_train)

In [ ]:
x_test_scale = ss.transform(X_test)

lda.score(x_test_scale, y_test)

> And the same prediction methods from other classifiers

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wine.iloc[:, 1:],
                                                   wine.iloc[:, 0],
                                                   test_size=0.20
                                                   )

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

p = Pipeline([('scaling', StandardScaler()), 
              ('lda', LDA()),
              ('model', LogisticRegression(solver='liblinear'))
             ])

params = {'model__C': [0.001, 0.1, 1, 10], 'lda__n_components': [1, 2]}

wine_search = GridSearchCV(p, param_grid=params, scoring='accuracy', cv=10, refit=True)
wine_search = wine_search.fit(X_train, y_train)

wine_search.best_estimator_

In [ ]:
print(f'Validation score: {wine_search.best_score_:.2%}')
print(f'Test score: {wine_search.score(X_test, y_test):.2%}')

> Near perfect accuracy on the `wine` data, though PCA had similar performance, albeit with more components.

# [Manifold Learning](https://scikit-learn.org/stable/modules/manifold.html)
Approach for nonlinear dimension reduction, but mainly used for visualizing data.  

<img src='./diagrams/manifold.png'>

[Image source scikit-learn manifold](https://scikit-learn.org/stable/modules/manifold.html#manifold)

- Looks for how examples relate linearly to their closest neighbors.  
- Looks for lower dimensions that preserve those relations.  
- Very common for visualizing higher dimension data.
- Here's a [short video](https://www.youtube.com/watch?v=wvsE8jm1GzE) that describes the concept at a high-level.  
- Most common use-case for these is visualization. See [the embedding projector](https://projector.tensorflow.org/) for an interesting example.

## t-SNE (t-Distributed Stochastic Neighbor Embedding)
From [sci-kit learn](https://scikit-learn.org/stable/modules/manifold.html):
>t-SNE (TSNE) converts affinities of data points to probabilities. The affinities in the original space are represented by Gaussian joint probabilities and the affinities in the embedded space are represented by Student’s t-distributions. This allows t-SNE to be particularly sensitive to local structure and has a few other advantages over existing techniques:

> - Better at revealing the structure at many scales on a single map  
> - Better at revealing data that lie in multiple, different, manifolds or clusters  
> - Better at reducing the tendency to crowd points together at the center  
> - While Isomap, LLE and variants are best suited to unfold a single continuous low dimensional manifold, t-SNE will focus on the local structure of the data and will tend to extract clustered local groups of samples as highlighted on the S-curve example. This ability to group samples based on the local structure might be beneficial to visually disentangle a dataset that comprises several manifolds at once as is the case in the digits dataset.

> The Kullback-Leibler (KL) divergence of the joint probabilities in the original space and the embedded space will be minimized by gradient descent. Note that the KL divergence is not convex, i.e. multiple restarts with different initializations will end up in local minima of the KL divergence. Hence, it is sometimes useful to try different seeds and select the embedding with the lowest KL divergence.

The disadvantages to using t-SNE are roughly:

- t-SNE is computationally expensive, and can take several hours on million-sample datasets where PCA will finish in seconds or minutes  
- The Barnes-Hut t-SNE method is limited to two or three dimensional embeddings.  
- The algorithm is stochastic and multiple restarts with different seeds can yield different embeddings. However, it is perfectly legitimate to pick the embedding with the least error.  
- Global structure is not explicitly preserved. This problem is mitigated by initializing points with PCA (using init='pca').

[See *How to Use t-SNE Effectively* for best practices and tuning tips.](https://distill.pub/2016/misread-tsne/)

### Math Behind SNE (Stochastic Neighbor Embedding)
[See Maaten's paper overviewing the technique](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf)

- Convert the Euclidean distances into conditional probabilities that represent similarities. Similarity of $x_j$ to $x_i$ is the conditional probability that $x_i$ would pick $x_j$ as one of its neighbors. Neighbors are picked according to their probability density, centered at $x_i$.
- $p_{j|i}$ will be high for nearby points and approach $0$ for faraway points.  

$$
p_{j|i}=\frac{exp(-||x_i-x_j||^2/2\sigma_i^2)}{\sum_{k\neq i}exp(-||x_i-x_k||^2/2\sigma_i^2)}
$$

Where $\sigma_i$ is the variance of the Gaussian centered at x_i. 

- $x_i$ and $x_j$ have low dimension counterparts, $y_i$ and $y_j$, which have a similar probability distribution:

$$
q_{j|i}=\frac{exp(-||y_i-y_j||^2)}{\sum_{k\neq i}exp(-||y_i-y_k||^2)}
$$

- $\sigma$ is set to $\frac{1}{(\sqrt2)}$, so that term disappears in $q_{j|i}$  
- If the data is perfectly projected $p_{j|i}=q_{j|i}.  
- It won't be perfect, so we look to minimize the difference, which is called the Kullback-Leibler divergences. This is done via gradient descent. The cost function is:

$$
C = \sum KL(P_i||Q_i) = \sum_i \sum_j p_{j|i} log \frac{p_{j|i}} {q_{j|i}}
$$

#### SNE is hard to optimize, since \sigma needs to be estimated (see paper).
t-SNE instead employs a Student t-distribution, which leads to:

$$
q_{ij}=\frac{(1+||y_i-y_j||^2)^{-1}}{\sum_{k \neq 1} (1+||y_k-y_l||^2)^{-1}}
$$

And the gradients become easier to calculate:

$$
\frac{\delta C}{\delta y_i} = 4\sum_{j}(p_{ij}-q_{ij})(y_i-y_j)(1+||y_i-y_j||^2)^{-1}
$$


The [linked paper](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf) above goes a bit more into the math if you are interested.

#### Attempting to use PCA to visualize the MNIST data

In [ ]:
from sklearn.decomposition import PCA
import datetime

ts_start = datetime.datetime.now()
pca = PCA(n_components=2).fit_transform(mnist_sample)

ts_end = datetime.datetime.now()

print(f'Completed in {ts_end-ts_start}')

pca_df = pd.DataFrame(pca)
pca_df.index = mnist_sample.index.tolist()
pca_df.columns = ['component1', 'component2']

pca_df = pd.concat([pca_df, mnist_sample_targets], axis=1)

import seaborn as sns

sns.set(rc={'figure.figsize':(9,9)})
sns.scatterplot(x='component1', y='component2', hue='class', data=pca_df)
plt.show()

> PCA doesn't visually separate the data well.

#### t-SNE

In [ ]:
from sklearn.manifold import TSNE
import datetime

ts_start = datetime.datetime.now()
tsne_model = TSNE(n_components=2, 
            init='random', 
            learning_rate='auto', 
            perplexity=30)

tsne = tsne_model.fit_transform(mnist_sample)

ts_end = datetime.datetime.now()

print(f'Completed in {ts_end-ts_start}')

tsne_df = pd.DataFrame(tsne)
tsne_df.index = mnist_sample.index.tolist()
tsne_df.columns = ['component1', 'component2']

tsne_df = pd.concat([tsne_df, mnist_sample_targets], axis=1)

import seaborn as sns

sns.set(rc={'figure.figsize':(9,9)})
sns.scatterplot(x='component1', y='component2', hue='class', data=tsne_df)
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import datetime

ts_start = datetime.datetime.now()
tsne = TSNE(n_components=2, 
            init='random', 
            learning_rate='auto', 
            perplexity=5).fit_transform(mnist_sample)

ts_end = datetime.datetime.now()

print(f'Completed in {ts_end-ts_start}')

tsne_df = pd.DataFrame(tsne)
tsne_df.index = mnist_sample.index.tolist()
tsne_df.columns = ['component1', 'component2']

tsne_df = pd.concat([tsne_df, mnist_sample_targets], axis=1)

import seaborn as sns

sns.set(rc={'figure.figsize':(9,9)})
sns.scatterplot(x='component1', y='component2', hue='class', data=tsne_df)
plt.show()

- Cluster size isn't meaningful. 
- May change shape given hyperparameter settings.  
- Cluster closeness may not be meaningful.  

# Readings

- [Machine Learning with Python 3rd Edition, Chapter 5](https://github.com/rasbt/python-machine-learning-book-3rd-edition/tree/master/ch05)  
- [Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow, Chapter 8](https://github.com/ageron/handson-ml2/blob/master/08_dimensionality_reduction.ipynb)
- [t-SNE Wiki](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)  
- [t-SNE Papers by Laurens van der Maaten](https://lvdmaaten.github.io/tsne/)
- [What, Why and How of t-SNE](https://towardsdatascience.com/what-why-and-how-of-t-sne-1f78d13e224d)